In [1]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
import pandas as pd
import numpy as np
import string
import re


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df=pd.read_csv('SEresult.csv')
df.head(10)

,post.ID,Caption,Likes
0,3110459107111173370,Berdasarkan survei elektabilitas capres yang d...,5748
1,Username,Comment,NaN
2,aditya.sanchez,Karena Anies bisa menang di Jakarta saja karen...,NaN
3,radenadindaaa,"Posisi ketiga terus, tapi kayaknya dijegal ter...",NaN
4,imamdjamaludin,Hasil survey litbang kompas sebelum pilkada dk...,NaN
5,fajar_ghosyiah,"Heh,wowkwowk waktu pilgub jakarta juga peringk...",NaN
6,sigitpurwaginanjar99,Menang survei kok bangga.. liat aja dilapangan...,NaN
7,2ndndnd_,"pernyataan seperti ini, jadi keinget sama pern...",NaN
8,lagimanberly,Tiga aja di jegal di takuti. Bagaimana kalau n...,NaN
9,marifatullahmy,Lah emang konsep perubahan yang di tawarkan ol...,NaN


In [6]:
df = df.iloc[5:]
df = df.drop(['post.ID','Likes'], axis =1)
df.head(10)

,Caption
5,"Heh,wowkwowk waktu pilgub jakarta juga peringk..."
6,Menang survei kok bangga.. liat aja dilapangan...
7,"pernyataan seperti ini, jadi keinget sama pern..."
8,Tiga aja di jegal di takuti. Bagaimana kalau n...
9,Lah emang konsep perubahan yang di tawarkan ol...
10,Sampai skrg saya bingung apa ide dan gagasan p...
11,Waktu mendekati pilkada DKI juga posisi elekta...
12,Karna pak anis kaga punya duit buat nyewa lemb...
13,+/- 25 % yang dukung anies punya kepentingan ...
14,Kalo emang survei elektabilitas ini bisa jadi ...


In [7]:
print('Case Folding Result : \n')
print(df['Caption'].head(5))
print('\n\n\n')

Case Folding Result : 

5    Heh,wowkwowk waktu pilgub jakarta juga peringk...
6    Menang survei kok bangga.. liat aja dilapangan...
7    pernyataan seperti ini, jadi keinget sama pern...
8    Tiga aja di jegal di takuti. Bagaimana kalau n...
9    Lah emang konsep perubahan yang di tawarkan ol...
Name: Caption, dtype: object






In [8]:

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_special(text):
    text = str(text)
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    text = text.lower()
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Caption'] = df['Caption'].apply(remove_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Caption'] = df['Caption'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['Caption'] = df['Caption'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Caption'] = df['Caption'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Caption'] = df['Caption'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Caption'] = df['Caption'].apply(remove_singl_char)

# NLTK word rokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tokenized'] = df['Caption'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(df['tokenized'].head())
print('\n\n\n')

Tokenizing Result : 

5    [hehwowkwowk, waktu, pilgub, jakarta, juga, pe...
6    [menang, survei, kok, bangga, liat, aja, dilap...
7    [pernyataan, seperti, ini, jadi, keinget, sama...
8    [tiga, aja, di, jegal, di, takuti, bagaimana, ...
9    [lah, emang, konsep, perubahan, yang, di, tawa...
Name: tokenized, dtype: object






In [9]:
def freqDist_wrapper(text):
    return FreqDist(text)

df['freqdist'] = df['tokenized'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(df['freqdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

5    [(jadi, 2), (hehwowkwowk, 1), (waktu, 1), (pil...
6    [(menang, 1), (survei, 1), (kok, 1), (bangga, ...
7    [(pernyataan, 2), (juga, 2), (waktu, 2), (ga, ...
8    [(di, 2), (tiga, 1), (aja, 1), (jegal, 1), (ta...
9    [(lah, 1), (emang, 1), (konsep, 1), (perubahan...
Name: freqdist, dtype: object


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords

#stopword indonesia
list_stopwords = stopwords.words('indonesian')


# menggabungkan additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','awokawok','b aja','yang','jg'])


# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))


# memasukan list stopwords ke dalam corpus
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['tokens_WSW'] = df['tokenized'].apply(stopwords_removal)


print(df['tokens_WSW'].head())

5      [hehwowkwowk, pilgub, jakarta, peringkat, mulu]
6    [menang, survei, bangga, liat, dilapangan, mas...
7    [pernyataan, keinget, pernyataan, pakar, media...
8    [jegal, takuti, no, kejang, insya, allah, anie...
9    [emang, konsep, perubahan, tawarkan, ganjar, p...
Name: tokens_WSW, dtype: object


In [12]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['normalized'] = df['tokens_WSW'].apply(normalized_term)

df['normalized'].head(10)

5       [hehwowkwowk, pilgub, jakarta, peringkat, mulu]
6     [menang, survei, bangga, lihat, dilapangan, ma...
7     [pernyataan, keinget, pernyataan, pakar, media...
8     [jegal, takuti, no, kejang, insya, allah, anie...
9     [emang, konsep, perubahan, tawarkan, ganjar, p...
10    [sekarang, bingung, ide, gagasan, perubahan, a...
11    [mendekati, pilkada, dki, posisi, elektabilita...
12    [karna, anies, tidak, duit, nyewa, lembaga, su...
13    [dukung, anies, kepentingan, berpendidikan, ke...
14    [emang, survei, elektabilitas, acuan, hasil, a...
Name: normalized, dtype: object

In [13]:
normalizad_word_dict

{'skrg': 'sekarang',
 'lu': 'kamu',
 'luh': 'kamu',
 'loe': 'kamu',
 'lo': 'kamu',
 'emg': 'memang',
 'dr': 'dari',
 'gw': 'saya',
 'tpi': 'tetapi',
 'tp': 'tetapi',
 'krn': 'karena',
 'anis': 'anies',
 'blom': 'belum',
 'blm': 'belum',
 'sopo': 'siapa',
 'kaga': 'tidak',
 'gua': 'saya',
 'anda': 'kamu',
 'liat': 'lihat',
 'aje': 'saja',
 'doi': 'dia',
 'surpay': 'survey',
 'surpey': 'survey',
 'bngt': 'banget',
 'surpaii': 'survey',
 'syurrrpaii': 'survey',
 'mkin': 'semakin',
 'mlh': 'malah',
 'prabwo': 'prabowo',
 'koq': 'kok',
 'lbih': 'lebih'}

In [14]:
normalizad_word.head()

,before,after
0,skrg,sekarang
1,lu,kamu
2,luh,kamu
3,loe,kamu
4,lo,kamu


In [15]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("===")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['tokens_stemmed'] = df['normalized'].swifter.apply(get_stemmed_term)
print(df['tokens_stemmed'])

891
hehwowkwowk : hehwowkwowk
pilgub : pilgub
jakarta : jakarta
peringkat : peringkat
mulu : mulu
menang : menang
survei : survei
bangga : bangga
lihat : lihat
dilapangan : lapang
masyarakat : masyarakat
cerdas : cerdas
pernyataan : nyata
keinget : keinget
pakar : pakar
media : media
pesimis : pesimis
formula : formula
digelar : gelar
jangka : jangka
tinggal : tinggal
menikmati : nikmat
proses : proses
tanggal : tanggal
mainnya : main
kompetisi : kompetisi
gegabah : gegabah
matang : matang
dipikirkan : pikir
mendukung : dukung
anies : anies
ikutan : ikut
satupun : satu
milih : milih
karna : karna
buang : buang
jegal : jegal
takuti : takut
no : no
kejang : kejang
insya : insya
allah : allah
presiden : presiden
emang : emang
konsep : konsep
perubahan : ubah
tawarkan : tawar
ganjar : ganjar
prabowo : prabowo
sekarang : sekarang
bingung : bingung
ide : ide
gagasan : gagas
berkutat : kutat
menyinggung : singgung
program : program
jokowi : jokowi
diusung : usung
mendekati : dekat
pilkada : p

Pandas Apply:   0%|          | 0/398 [00:00<?, ?it/s]

5        [hehwowkwowk, pilgub, jakarta, peringkat, mulu]
6      [menang, survei, bangga, lihat, lapang, masyar...
7      [nyata, keinget, nyata, pakar, media, pesimis,...
8      [jegal, takut, no, kejang, insya, allah, anies...
9          [emang, konsep, ubah, tawar, ganjar, prabowo]
                             ...                        
398                                       [pange, pange]
399                       [kerja, beres, lihat, jakarta]
400                        [hahahha, survei, abal, abal]
401                              [tenang, menjawabanies]
402    [konsep, ubah, visi, belum, track, record, men...
Name: tokens_stemmed, Length: 398, dtype: object


In [17]:
df['merged'] = df['tokens_stemmed'].apply(lambda x: ' '.join(x))

In [18]:
df.head(10)

,Caption,tokenized,freqdist,tokens_WSW,normalized,tokens_stemmed,merged
5,hehwowkwowk waktu pilgub jakarta juga peringka...,"[hehwowkwowk, waktu, pilgub, jakarta, juga, pe...","{'hehwowkwowk': 1, 'waktu': 1, 'pilgub': 1, 'j...","[hehwowkwowk, pilgub, jakarta, peringkat, mulu]","[hehwowkwowk, pilgub, jakarta, peringkat, mulu]","[hehwowkwowk, pilgub, jakarta, peringkat, mulu]",hehwowkwowk pilgub jakarta peringkat mulu
6,menang survei kok bangga liat aja dilapangan m...,"[menang, survei, kok, bangga, liat, aja, dilap...","{'menang': 1, 'survei': 1, 'kok': 1, 'bangga':...","[menang, survei, bangga, liat, dilapangan, mas...","[menang, survei, bangga, lihat, dilapangan, ma...","[menang, survei, bangga, lihat, lapang, masyar...",menang survei bangga lihat lapang masyarakat c...
7,pernyataan seperti ini jadi keinget sama perny...,"[pernyataan, seperti, ini, jadi, keinget, sama...","{'pernyataan': 2, 'seperti': 1, 'ini': 1, 'jad...","[pernyataan, keinget, pernyataan, pakar, media...","[pernyataan, keinget, pernyataan, pakar, media...","[nyata, keinget, nyata, pakar, media, pesimis,...",nyata keinget nyata pakar media pesimis formul...
8,tiga aja di jegal di takuti bagaimana kalau no...,"[tiga, aja, di, jegal, di, takuti, bagaimana, ...","{'tiga': 1, 'aja': 1, 'di': 2, 'jegal': 1, 'ta...","[jegal, takuti, no, kejang, insya, allah, anie...","[jegal, takuti, no, kejang, insya, allah, anie...","[jegal, takut, no, kejang, insya, allah, anies...",jegal takut no kejang insya allah anies presiden
9,lah emang konsep perubahan yang di tawarkan ol...,"[lah, emang, konsep, perubahan, yang, di, tawa...","{'lah': 1, 'emang': 1, 'konsep': 1, 'perubahan...","[emang, konsep, perubahan, tawarkan, ganjar, p...","[emang, konsep, perubahan, tawarkan, ganjar, p...","[emang, konsep, ubah, tawar, ganjar, prabowo]",emang konsep ubah tawar ganjar prabowo
10,sampai skrg saya bingung apa ide dan gagasan p...,"[sampai, skrg, saya, bingung, apa, ide, dan, g...","{'sampai': 2, 'skrg': 2, 'saya': 1, 'bingung':...","[skrg, bingung, ide, gagasan, perubahan, anies...","[sekarang, bingung, ide, gagasan, perubahan, a...","[sekarang, bingung, ide, gagas, ubah, anies, s...",sekarang bingung ide gagas ubah anies sekarang...
11,waktu mendekati pilkada dki juga posisi elekta...,"[waktu, mendekati, pilkada, dki, juga, posisi,...","{'waktu': 1, 'mendekati': 1, 'pilkada': 1, 'dk...","[mendekati, pilkada, dki, posisi, elektabilita...","[mendekati, pilkada, dki, posisi, elektabilita...","[dekat, pilkada, dki, posisi, elektabilitas, a...",dekat pilkada dki posisi elektabilitas anies u...
12,karna pak anis kaga punya duit buat nyewa lemb...,"[karna, pak, anis, kaga, punya, duit, buat, ny...","{'karna': 1, 'pak': 1, 'anis': 1, 'kaga': 1, '...","[karna, anis, kaga, duit, nyewa, lembaga, surv...","[karna, anies, tidak, duit, nyewa, lembaga, su...","[karna, anies, tidak, duit, nyewa, lembaga, su...",karna anies tidak duit nyewa lembaga survey mi...
13,yang dukung anies punya kepentingan memang ga ...,"[yang, dukung, anies, punya, kepentingan, mema...","{'yang': 1, 'dukung': 1, 'anies': 1, 'punya': ...","[dukung, anies, kepentingan, berpendidikan, ke...","[dukung, anies, kepentingan, berpendidikan, ke...","[dukung, anies, penting, didik, kena, tipu]",dukung anies penting didik kena tipu
14,kalo emang survei elektabilitas ini bisa jadi ...,"[kalo, emang, survei, elektabilitas, ini, bisa...","{'kalo': 1, 'emang': 1, 'survei': 1, 'elektabi...","[emang, survei, elektabilitas, acuan, hasil, a...","[emang, survei, elektabilitas, acuan, hasil, a...","[emang, survei, elektabilitas, acu, hasil, aho...",emang survei elektabilitas acu hasil ahok udah...


In [19]:
df.to_csv("processed.csv")